In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
import pandas as pd
import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.keys import Keys
from collections import namedtuple
from bs4 import BeautifulSoup

In [2]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
website = 'https://www.desaparecidos.pr.gov.br/desaparecidos/desaparecidos.do?action=iniciarProcesso&m=true'
driver.get(website)

driver.maximize_window()

In [3]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [4]:
nomes = []
local_desaparecimento = []
idade_atual = []

def scrape_website(url):
    driver.get(url)  # Carrega a URL no navegador controlado pelo Selenium

    current_page = 1
    try:
        while True:
            time.sleep(5)  # Aguarde a página carregar completamente após cada clique
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            
            divs = soup.find_all('div', class_='col-md-6 mb-3')
            for div in divs:
                col_md_9 = div.find('div', class_='col-md-9')
                if col_md_9:
                    strong_tags = col_md_9.find_all('strong')
                    if len(strong_tags) >= 6:
                        nomes.append(strong_tags[0].get_text(strip=True))
                        local_desaparecimento.append(strong_tags[2].next_sibling.strip())
                        idade_atual.append(strong_tags[4].next_sibling.strip())
            
            # Tentar navegar para a próxima página
            current_page += 1
            try:
                next_page_link = driver.find_element(By.XPATH, f"//a[@href='javascript:pesquisar({current_page});']")
                next_page_link.click()
            except:
                print("Última página alcançada ou link não encontrado.")
                break
    except Exception as e:
        print("Erro durante a navegação:", e)

try:
    # URLs para percorrer
    urls = [
        'https://www.desaparecidos.pr.gov.br/desaparecidos/desaparecidos.do?action=iniciarProcesso&m=true'
    ]

    for url in urls:
        scrape_website(url)

finally:
    driver.quit()  # Fechar o navegador ao finalizar

Última página alcançada ou link não encontrado.


In [8]:
data = {
    'Nomes': nomes,
    'Idades': idade_atual,
    'Local de Desaparecimento': local_desaparecimento
}

df = pd.DataFrame(data)
df.to_csv('Desaparecidos Polícia Civil PR - crianças.csv', index=False)